# Project Sparkify
Sparkify is a music streaming startup with an impressive userbase growth _(their marketing team must be doing one hell of a job)_ and is looking to move their processes and data to the cloud. Their data resides in S3, in a directory of JSON logs on user activity on the app, as well as a directory with JSON metadata on the songs in their app.

I am tasked with building a data warehouse for the analytics team, I will basically build an ETL pipeline that extracts their data from S3, stages them in Redshift, and transform the data into a set of dimensional tables for their analytics team to continue finding insights in what songs their users are listening to.

Let's get to work:

In [56]:
# First things first, we have to create a redshift cluster for our project on AWS
# Here, we'd be using IaC to proceed with the processes

# importing boto3, AWS python SDK
import boto3

import configparser
import json
import pandas as pd

In [38]:
# Extracting config variables 
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY = config.get('USER', 'KEY')
SECRET = config.get('USER', 'SECRET')

In [40]:
DWH_ROLE_NAME = config.get('CLUSTER', 'DWH_ROLE_NAME')
DWH_DB_NAME = config.get('CLUSTER', 'DWH_DB_NAME')
DWH_CLUSTER_ID = config.get('CLUSTER', 'DWH_CLUSTER_ID')
DWH_NODE_TYPE = config.get('CLUSTER', 'DWH_NODE_TYPE')
DWH_USER_NAME = config.get('CLUSTER', 'DWH_USER_NAME')
DWH_USER_PASSWORD = config.get('CLUSTER', 'DWH_USER_PASSWORD')
DWH_NUMBER_0F_NODES = int(config.get('CLUSTER', 'DWH_NUMBER_0F_NODES'))

variables = pd.DataFrame({
    'keys':['DWH_ROLE_NAME', 'DWH_DB_NAME', 'DWH_CLUSTER_ID', 'DWH_NODE_TYPE', 'DWH_NUMBER_0F_NODES'], 
    'values':[DWH_ROLE_NAME, DWH_DB_NAME, DWH_CLUSTER_ID, DWH_NODE_TYPE, DWH_NUMBER_0F_NODES]
})

variables

,keys,values
0,DWH_ROLE_NAME,redshift_s3_readonly
1,DWH_DB_NAME,sparkifydb
2,DWH_CLUSTER_ID,sparkify-cluster
3,DWH_NODE_TYPE,dc2.large
4,DWH_NUMBER_0F_NODES,4


In [18]:
# Next we create an IAM role for our redshift clust with AmazonS3ReadOnlyAccess permision
iam = boto3.client('iam', region_name='us-east-2', aws_access_key_id=KEY, aws_secret_access_key=SECRET)

try:
    print('Creating IAM role for Redshift cluster...')
    iam_role = iam.create_role(
        RoleName=DWH_ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps({
            'Statement': [{
                'Action': 'sts:AssumeRole',
                'Effect': 'Allow',
                'Principal': {'Service': 'redshift.amazonaws.com'}
            }],
            'Version': '2012-10-17'
        }),
        Description='Allows Redshift cluster to call AWS services on you behalf',
    )
    print('Role creation successful!')
except Exception as e:
    print(e)

Creating IAM role for Redshift cluster...
Role creation successful!


In [17]:
iam.attach_role_policy(
    RoleName=DWH_ROLE_NAME,
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
)['ResponseMetadata']['HTTPStatusCode']

200

In [1]:
role_arn = iam_role['Role']['Arn']
role_arn

In [3]:
# Now we build the redshift cluster
redshift = boto3.client('redshift', region_name='us-east-2', aws_access_key_id=KEY, aws_secret_access_key=SECRET)

try:
    print('Creating Redshift cluster...')
    redshift_cluster = redshift.create_cluster(
        DBName=DWH_DB_NAME,
        ClusterIdentifier=DWH_CLUSTER_ID,
        NodeType=DWH_NODE_TYPE,
        MasterUsername=DWH_USER_NAME,
        MasterUserPassword=DWH_USER_PASSWORD,
        NumberOfNodes=DWH_NUMBER_0F_NODES,
        IamRoles=[
            role_arn,
        ]
    )
    print('Redshift cluster creation successful!')
except Exception as e:
    print(e)

In [5]:
# Checking cluster availability status
cluster_props = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_ID)['Clusters'][0]
cluster_props['ClusterAvailabilityStatus']

In [6]:
response = redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_ID, SkipFinalClusterSnapshot=True)
response